In [ ]:
from PYME.LMVis import VisGUI

%gui wx

In [ ]:
pymevis = VisGUI.ipython_pymevisualize()
pipeline = pymevis.pipeline

In [ ]:
import numpy as np
from PYME.IO import tabular


##### PARAMETERS to Change (see 'cluster_analysis_V1' function below for more detailed descriptions of parameters)

min_dist = 10       # minimum distance between skeleton vertices in determining the vector that points from each vertex
                    # to the closest vertex that's at least this distance away (helps make nice backbone vectors)
max_dist = 50       # maximum distance between skeleton vertices. This helps speed up the KDTree query
max_nn = 200        # maximum # of nearest neighbors on the skeleton before they are removed
ball_r = 50         # radius used for query_ball_points
Rtn4_chan = 'chan1' # channel name for Rtn4 data
high_filt=100          # Threshold for max distance a Rtn4 point can be from mesh and still be included in analysis
low_filt=-100          # Threshold for min distance a Rtn4 point can be from mesh and still be included in analysis
min_size = 10      # currently not used
max_size = 100     # currently not used
rg_factor = 0.925  # Factor to adjust radius of gyration for simulations
savedir = "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_analysis_V2\\updated_analysis_20230111"
ver = 'V1-adj'

datadir = ["K:\\4Pi_data\Oligomer_analysis\\Oligomer_data\\20210907_data\\Cell02_DBSCAN-2_ROI_all-clumped_DBSCAN-clusters_measureClusters3D.csv",
           "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20220930_data\\Cell04_ROI1_all-clumped_MeasureCluster3D_DBSCAN.csv",
           "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20221004_data\\Cell05_ROI4_all-clumped_measureClusters3D.csv"]
c_points = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Cell02_DBSCAN-2_ROI_all-clumped_DBSCAN-clusters.hdf",
            "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20220930_data\\Cell04_ROI1_all-clumped_DBSCAN_Clusters.hdf",
            "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20221004_data\\Cell05_ROI4_all-clumped_DBSCAN_Clusters.hdf"]
mesh_fn = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Cell02_DBSCAN-2_ROI_shrinkwrap.stl",
               "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20220930_data\\Cell04_ROI1_shrinkwrap_1.stl",
               "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20221004_data\\Cell05_ROI4_shrinkwrap.stl"]
skelly = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Cell02_DBSCAN-2_ROI_skeleton.stl",
          "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20220930_data\\Cell04_ROI1_skeleton.stl",
          "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20221004_data\\Cell05_ROI4_skeleton.stl"]
loc_info = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Cell02_DBSCAN-2_ROI_all-clumped_dbscanID_locs.csv",
            "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20220930_data\\Cell04_ROI1_all_clumped_dbscanID_locs.csv",
            "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20221004_data\\Cell05_ROI4_all-clumped_dbscanID_locs.csv"]

d_name = ['20210907_Cell02','20220930_Cell04','20221004_Cell05']

In [ ]:
# For figure data:
# datadir = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Old_results\\Cell02_oligomer_ROI2_DBSCAN.csv"]
# c_points = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Old_results\\Cell02_oligomer_ROI2_DBSCAN.hdf"]
# mesh_fn = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Old_results\\Cell02_oligomer_ROI2_shrinkwrap.stl"]
# skelly = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Old_results\\Cell02_oligomer_ROI2_skeleton_V2.stl"]
# loc_info = ["K:\\4Pi_data\\Oligomer_analysis\\Oligomer_data\\20210907_data\\Old_results\\Cell02_mapped_clumped_ROI2_dbscanID_locs.csv"]
# ver = 'test_data'

In [ ]:
params = {'min_dist': min_dist, 'max_dist': max_dist, 'max_nn': max_nn, 'ball_r': ball_r, 'Rtn4_chan': Rtn4_chan,
              'high_filt': high_filt, 'low_filt': low_filt, 'min_size': min_size,
              'max_size': max_size, 'savedir': savedir, 'ver': ver, 'datadir': datadir, 'c_points': c_points,
              'mesh_fn': mesh_fn, 'skelly': skelly, 'loc_info': loc_info, 'd_name': d_name}

In [ ]:
import numpy as np
from PYME.IO import tabular
from scipy.spatial import cKDTree

# generate 3D Gaussian
# points = np.random.randn(100,3)*100

#######################################################################################################################
# Define needed functions
#######################################################################################################################

def add_ds_from_Nx3(points, pipeline, pymevis, ds_name='points', color=None, normals=None):
    """
    Quickly add points/normals to the pipeline and pymevis display.
    """
    
    d = {'x': points[:,0], 'y': points[:,1], 'z': points[:,2]}
    
    if color is not None:
        d['c'] = color
        
    if normals is not None:
        d['xn'] = normals[:,0]
        d['yn'] = normals[:,1]
        d['zn'] = normals[:,2]

    # create tabular mappingFilter data source and add it to pymevis
    pipeline.addDataSource(ds_name, tabular.mappingFilter(d))

    # select this data source (optional, but helps support "default behavior")
    pipeline.selectDataSource(ds_name)

    # Add a pointcloud layer that displays data source named 'points' (the one we just added)
    pymevis.add_pointcloud_layer(ds_name=ds_name)
    
    if normals is not None:
        pymevis.glCanvas.layers[-1].display_normals=True
        pymevis.glCanvas.layers[-1].normal_scaling=25.0

def add_mesh_ds(vertices, param, pipeline, pymevis, param_name='new_param', ds_name='new_param', color=None, normals=None):
    """
    Quickly add surface that has information about distance to another surface
    """
    
    #d = {'vertices': vertices, 'faces': faces, 'surf-surf-distance': surf_dist}
    d = {'x': vertices[:,0], 'y': vertices[:,1], 'z': vertices[:,2]} 

    if color is not None:
        d['c'] = color
        
    if normals is not None:
        d['xn'] = normals[:,0]
        d['yn'] = normals[:,1]
        d['zn'] = normals[:,2]

    # create tabular mappingFilter data source and add it to pymevis
    out = tabular.MappingFilter(d)
    out.addColumn(param_name, param)
    pipeline.addDataSource(ds_name, out)                    

    # select this data source (optional, but helps support "default behavior")
    pipeline.selectDataSource(ds_name) 
        
def save_snapshot(canvas, file_name=None):
    if True:
        pixel_size=None
        
        if file_name is None:
            file_name = wx.FileSelector('Save current view as', wildcard="PNG files(*.png)|*.png",
                            flags=wx.FD_SAVE | wx.FD_OVERWRITE_PROMPT)
        
        if file_name:
            snap = canvas.getIm(pixel_size, GL_RGB)
            print(snap.dtype, snap.shape, snap.max())
            if snap.ndim == 3:
                img = PIL.Image.fromarray(snap.transpose(1, 0, 2))
                #img = toimage(snap.transpose(1, 0, 2))
            else:
                img = PIL.Image.fromarray(snap.transpose())
                #img = toimage(snap.transpose())
            
            img = img.transpose(PIL.Image.FLIP_TOP_BOTTOM)
            
            if not file_name.endswith('.png'):
                img.save('{}.png'.format(file_name))
            else:
                img.save('{}'.format(file_name))

def fullprint(*args, **kwargs):
  from pprint import pprint
  import numpy
  opt = numpy.get_printoptions()
  numpy.set_printoptions(threshold=numpy.inf)
  pprint(*args, **kwargs)
  numpy.set_printoptions(**opt)
    
def tan_circ_sim(rad, count, loc_prec, rg_factor, centers, surf_verts, skeleton, vecs_hat):
    """
    Simulate clusters that are disc-shaped, tangent to the surface, and centered at the true cluster centers.
    They also contain the same number of simulated localizations as the real cluster they are based on.
    
    Followed advice for determining plane for cicle from:
    https://stackoverflow.com/questions/71160423/how-to-sample-points-in-3d-in-python-with-origin-and-normal-vector
    
    Also followed advice for uniform random distributions on a circle from here:
    https://math.stackexchange.com/questions/1307287/random-uniformly-distributed-points-in-a-circle
    
    ------
    Input:
    ------
    
    rad (numpy array): radius of the simulated disc
    
    count (numpy array of ints): number of localizations to be simulated in each cluster
    
    -------
    Output:
    -------
    
    cs_points_all (numpy array): coordinates of points in the simulated clusters
    
    """
    from numpy import pi

    # parameters for this simulation
    #rad = radius # radius of points sampled in the circle on the plane (probably leave as 1 always)
    max_r = rad * rg_factor
    v_num = count # number of vectors desired (evenly spaced around the circle)
    lp = loc_prec
    rng = np.random.default_rng()

    # equations for making the vectors
    # rho = np.linspace(0, 2*np.pi, v_num)
    # x = np.cos(rho) * r
    # y = np.sin(rho) * r
    # z = np.zeros(rho.shape)
    # positions of simulated clusters
    arot_pos = centers
    # establish initial vectors
    tree_surf = cKDTree(surf_verts)
    mean_tree_skeleton = cKDTree(skeleton)
    arot_surf_dist, arot_surf_ind = tree_surf.query(arot_pos, k=1)
    arot_skel_dist, arot_skel_ind = mean_tree_skeleton.query(arot_pos, k=1)
    arot_skel_vec = arot_pos - skeleton[arot_skel_ind]
    arot_vecs = np.cross(vecs_hat[arot_skel_ind], arot_skel_vec)
    arot_tub = vecs_hat[arot_skel_ind]

    # Create vectors in a circle in the plane perpendicular to the vector that points from cluster center to skeleton
    f_all = []
    cs_points_sep = []
    cs_points_tog = []
    for bd in range(len(arot_pos)):
        rho = rng.uniform(0, 2*np.pi, int(v_num[bd]))
        uni = rng.uniform(0,1,int(v_num[bd]))
        p1 = arot_pos[bd]
        n = arot_skel_vec[bd] # vector normal to the plane
        #pp = arot_vecs[bd] #vector orthogonal to normal
        nabs = np.absolute(n)
        indices = (-n).argsort()[:3]
        v = np.zeros(3)
        v[indices[1]] = -n[indices[0]]
        v[indices[0]] = n[indices[1]]
        v[indices[2]] = 0
        u = np.cross(n,v)
        #normalize vectors
        v_norm = v/np.linalg.norm(v)
        u_norm = u/np.linalg.norm(u)
        f = []
        cs_points = []
        for be in range(int(v_num[bd])):
            r = np.sqrt(uni[be])*max_r[bd]
            # x = np.cos(rho[be]) * r
            # y = np.sin(rho[be]) * r
            # z = np.zeros(rho[be].shape)
            
            # Take the center point and add the uniform random location around the circle then add some localization precision
            unc = [rng.normal(loc=0,scale=lp,size=1),rng.normal(loc=0,scale=lp,size=1),rng.normal(loc=0,scale=lp,size=1)]
            circ_p = np.zeros(3)
            for bf in range(len(circ_p)):
                circ_p[bf] = p1[bf] + r * v_norm[bf] * np.cos(rho[be]) + r * u_norm[bf] * np.sin(rho[be]) + unc[bf]# + rng.normal(loc=0,scale=lp,size=1)
            circ_vec = p1-circ_p
            cs_points_tog.append(circ_p)
            cs_points.append(circ_p)
            f.append(circ_vec/np.linalg.norm(circ_vec))
        #print(cs_points_tog)
        cs_points = np.asarray(cs_points)
        cs_points_sep.append(cs_points)
        f = np.asarray(f)
        f_all.append(f)
    cs_points_tog = np.asarray(cs_points_tog)
    cs_points_sep = np.asarray(cs_points_sep)
    f_all = np.asarray(f_all)
    # print('search here')
    # fullprint(cs_points_tog)
    add_ds_from_Nx3(cs_points_tog, pipeline, pymevis, 'Disc-sim-clusters', normals=None)
    
    return cs_points_sep, cs_points_tog

def tan_line_sim(rad, count, loc_prec, rg_factor, centers, surf_verts, skeleton, vecs_hat):
    """
    Simulate clusters that are linearly-shaped, tangent to the surface, and centered at the true cluster centers.
    They also contain the same number of simulated localizations as the real cluster they are based on, and the
    same localization precision. The difference between these simulated clusters and the circular simulated
    clusters, is the angle of each point in the circular clusters is randomized whereas all the points
    in the linear clusters have the same angle. The radius varies for both.
    
    Followed advice for determining plane for cicle from:
    https://stackoverflow.com/questions/71160423/how-to-sample-points-in-3d-in-python-with-origin-and-normal-vector
    
    Also followed advice for uniform random distributions on a circle from here:
    https://math.stackexchange.com/questions/1307287/random-uniformly-distributed-points-in-a-circle
    
    ------
    Input:
    ------
    
    rad (numpy array): radius of the simulated disc
    
    count (numpy array of ints): number of localizations to be simulated in each cluster
    
    -------
    Output:
    -------
    
    cs_points_all (numpy array): coordinates of points in the simulated clusters
    
    """
    from numpy import pi

    # parameters for this simulation
    #rad = radius # radius of points sampled in the circle on the plane (probably leave as 1 always)
    max_r = rad * rg_factor
    v_num = count # number of vectors desired (evenly spaced around the circle)
    lp = loc_prec
    rng = np.random.default_rng()

    # equations for making the vectors
    # rho = np.linspace(0, 2*np.pi, v_num)
    # x = np.cos(rho) * r
    # y = np.sin(rho) * r
    # z = np.zeros(rho.shape)
    # positions of simulated clusters
    arot_pos = centers
    # establish initial vectors
    tree_surf = cKDTree(surf_verts)
    mean_tree_skeleton = cKDTree(skeleton)
    arot_surf_dist, arot_surf_ind = tree_surf.query(arot_pos, k=1)
    arot_skel_dist, arot_skel_ind = mean_tree_skeleton.query(arot_pos, k=1)
    arot_skel_vec = arot_pos - skeleton[arot_skel_ind]
    arot_vecs = np.cross(vecs_hat[arot_skel_ind], arot_skel_vec)
    arot_tub = vecs_hat[arot_skel_ind]

    # Create vectors in a circle in the plane perpendicular to the vector that points from cluster center to skeleton
    f_all = []
    ls_points_sep = []
    ls_points_tog = []
    for bd in range(len(arot_pos)):
        rho = rng.uniform(0, 2*np.pi, 1) * np.ones(int(v_num[bd]))
        uni = rng.uniform(-1,1,int(v_num[bd]))
        p1 = arot_pos[bd]
        n = arot_skel_vec[bd] # vector normal to the plane
        #pp = arot_vecs[bd] #vector orthogonal to normal
        nabs = np.absolute(n)
        indices = (-n).argsort()[:3]
        v = np.zeros(3)
        v[indices[1]] = -n[indices[0]]
        v[indices[0]] = n[indices[1]]
        v[indices[2]] = 0
        u = np.cross(n,v)
        #normalize vectors
        v_norm = v/np.linalg.norm(v)
        u_norm = u/np.linalg.norm(u)
        f = []
        ls_points = []
        for be in range(int(v_num[bd])):
            # if uni[be] < 0:
            #     r = np.sqrt(np.absolute(uni[be]))*max_r[bd]*(-1)
            # else:
            #     r = np.sqrt(uni[be])*max_r[bd]
            r = uni[be] * max_r[bd]
            
            # Take the center point and add the uniform random location around the circle then add some localization precision
            unc = [rng.normal(loc=0,scale=lp,size=1),rng.normal(loc=0,scale=lp,size=1),rng.normal(loc=0,scale=lp,size=1)]
            circ_p = np.zeros(3)
            for bf in range(len(circ_p)):
                circ_p[bf] = p1[bf] + r * v_norm[bf] * np.cos(rho[be]) + r * u_norm[bf] * np.sin(rho[be]) + unc[bf]# + rng.normal(loc=0,scale=lp,size=1)
            circ_vec = p1-circ_p
            ls_points_tog.append(circ_p)
            ls_points.append(circ_p)
            f.append(circ_vec/np.linalg.norm(circ_vec))
        #print(cs_points_tog)
        ls_points = np.asarray(ls_points)
        ls_points_sep.append(ls_points)
        f = np.asarray(f)
        f_all.append(f)
    ls_points_tog = np.asarray(ls_points_tog)
    ls_points_sep = np.asarray(ls_points_sep)
    f_all = np.asarray(f_all)
    
    add_ds_from_Nx3(ls_points_tog, pipeline, pymevis, 'Line-sim-clusters', normals=None)
    
    return ls_points_sep, ls_points_tog
    
def proj_u_onto_plane(u, n):
    ''' 
    Project vector u onto plane that is orthogonal to vector n
    ----------------------------------------------------------
    Input:
        u: vector to be projected onto plane
        n: vector normal to the plane
    Output:
        u_p: projection of vector u onto plane with normal n
    '''
    n_norm = np.sqrt(sum(n**2))
    proj_of_u_on_n = (np.dot(u, n)/n_norm**2)*n
    u_p = u - proj_of_u_on_n
    return u_p

def vector_projection(cp, s):
    '''
    Project vector cp onto vector s
    -------------------------------
    Input:
        cp: projection of cluster vectors c onto plane normal to r
        s: used skeleton vectors
    Output:
        a: component of cp that is parallel to s (vector projection of cp onto s)
        
    Followed: https://www.geeksforgeeks.org/vector-projection-using-python/
    '''
    s_norm = np.sqrt(sum(s**2))
    a = (np.dot(cp, s)/s_norm**2)*s
    return a
    
def angle_btwn_vectors(b, cp):
    '''
    Calculate angle(alpha) between vector b and vector cp
    ----------------------------------------------
    Input:
        cp: projection of cluster vectors c onto plane normal to r
        b: the component of cp orthogonal to the skeleton axis
    Output:
        alpha: angle between vector b and vector cp
    '''
    b_norm = np.sqrt(sum(b**2))
    cp_norm = np.sqrt(sum(cp**2))
    alpha = np.absolute(np.arccos(np.dot(b,cp)/(b_norm*cp_norm))) #absolute value is technically correct but not actually necessary because angle between b and cp is range[-90 to +90]
    return alpha

def handedness(r, b, a):
    '''
    Calculate handedness of cluster vectors
    ---------------------------------------
    Input:
        r: vector from the centers of clusters to the closest part of the skeleton 
            (vector rejections of centers to skeleton points as these aren't always 
            perpindicular but the rejections are)
        b: the component of cp orthogonal to the skeleton axis
        a: component of cp that is parallel to s (vector projection of cp onto s)
    Output:
        h (-1 or 1): reflecting left or right handedness of cluster major axes on the
            tubule
    '''
    rxb_norm = np.sqrt(sum((np.cross(r,b))**2))
    a_norm = np.sqrt(sum(a**2))
    sign = np.dot(np.cross(r,b),a)/(rxb_norm*a_norm)
    return sign

def cluster_angles(r, c, s):
    '''
    Input: these 3 vectors should all be the same size
        r: vector from the centers of clusters to the closest part of the skeleton 
            (vector rejections of centers to skeleton points as these aren't always 
            perpendicular but the rejections are)
        c: cluster major axis vectors
        s: used skeleton vectors
        
    Output:
        psi: angle between the vector projection of c onto a plane positioned on the 
            surface and normal to r and vector b which the component of c orthogonal 
            to the skeleton axis
    '''
    cp = proj_u_onto_plane(c, r)
    a = vector_projection(cp, s)
    b = cp - a
    alpha = angle_btwn_vectors(cp,b)
    psi = angle_btwn_vectors(cp,s)
    sign = handedness(r, b, a)
    signed_alpha = alpha*sign
    signed_psi = psi*sign
    return signed_alpha, signed_psi

In [ ]:
def cluster_analysis_V1(params):
    '''
    This function does a number of things:
    
    - Calculate the angle between cluster vectors projected onto the surface and 
        that vectors component that is orthogonal to the tubule axis (angle psi)
    
    - Simulate circular and linear clusters and compare their anisotropies to the 
        real data
    
    Input:
    
    - param_list: Python dictionary of all required parameters (created above in 
        the parameter cell)
        
        - min_dist (int or float): minimum distance between skeleton vertices. 
            Used for calculating skeleton vectors. They point from each skeleton 
            point to the closest point that's at least this distance away
            
        - max_dist (int or float): maximum distance between skeleton vertices. 
            This helps speed up the KDTree query
        
        - max_nn (int): maximum # of nearest neighbors on the skeleton before 
            they are removed. This effectively removes sheets from the skeleton
            which we intentionally exclude from analyses.
            
        - ball_r (int or float): radius used for query_ball_points. Used for
            trimming the skeleton along with max_nn.
            
        - Rtn4_chan ('chan0' or 'chan1'): which channel of the two-color 4Pi data 
            is the one corresponding to the Rtn4 localizations.
            
        - high_filt (int or float): Threshold for max distance a Rtn4 point can 
            be from mesh surface and still be included in analysis.
            
        - low_filt (int or float): Threshold for min distance a Rtn4 point can be 
            from mesh surface and still be included in analysis.
            
        - min_size (int): Threshold for minimum # of localizations a cluster must 
            possess to be included in the anlaysis.
            
        - max_size (int): Threshold for maximum # of localizations a cluster must 
            possess to be included in the anlaysis.
            
        - savedir (str): File path where all file outputs will be saved.
            
        - ver (str): This string is included in every saved file name to make 
            sure older files are not overwritten if that is not desired.
            
        - datadir (list): list of file paths to .csv files of each dataset containing
            the results for MeasureClusters3D performed on DBSCAN segmetned clusters
            
        - c_points (list): list of file paths to .hdf files containing the DBSCAN
            segmented Rtn4 clusters.
            
        - mesh_fn (list): list of file paths to .stl files containing the 
            shrink-wrapped surfaces of each dataset.
            
        - skelly (list): list of file paths to .stl files containing the
            skeletons of each dataset.
            
        - loc_info (list): list of file paths to .csv files containing all info
            about each localization, including the cluster they belong to.
            
        - d_name (list): list of strings that are the names of the datasets
        
    Output:
    
    '''
    
    # Read data except for axes which are more involved
    clust_data = []
    cluster_output_all = []
    circ_sims_all = []
    line_sims_all = []
    adj_sims_all = []
    for dt in range(len(params['datadir'])):
        c_data = np.genfromtxt(params['datadir'][dt], delimiter = ',')
        l_data = np.genfromtxt(loc_info[dt], delimiter = ',')
        clust = {"count": c_data[:,0], "x": c_data[:,1], "y": c_data[:,2], "z": c_data[:,3], "gyrationRadius": c_data[:,4], 
                "median_abs_deviation": c_data[:,5], "sigma0": c_data[:,9], "sigma1": c_data[:,10], "sigma2": c_data[:,11], 
                "sigma_x": c_data[:,12], "sigma_y": c_data[:,13], "sigma_z": c_data[:,14], "anisotropy": c_data[:,15], 
                "theta": c_data[:,16], "phi": c_data[:,17], "loc_prec": l_data[:,12]}

        #making save-folder if it doesn't exist (DON'T TOUCH THE NEXT 5 LINES)
        from pathlib import Path
        import os
        p = Path(params['savedir'])
        if not p.exists():
            os.mkdir(params['savedir'])
        workdir = os.getcwd()

        import pandas as pd
        import copy

        if 'check1' in locals():
            del check1
        if 'check2' in locals():
            del check2
        if 'x' in locals():
            del x
        if 'y' in locals():
            del y
        if 'z' in locals():
            del z

        # Initialize variables
        csv_file = pd.read_csv(params['datadir'][dt], sep=',', header=None)
        clusto = {"axis0": np.copy(csv_file[6][1:].values), "axis1": np.copy(csv_file[7][1:].values), 
                  "axis2": np.copy(csv_file[8][1:].values)}
        clust['axis0'] = np.zeros((len(clust['count']),3))
        clust['axis1'] = np.zeros((len(clust['count']),3))
        clust['axis2'] = np.zeros((len(clust['count']),3))


        # Convert 'axis0', 'axis1', and 'axis2' values to floats from strings
        # axis0
        for a in range(len(clusto["axis0"])):
            #print('a:',a)
            if a > 0:
                del x,y,z,check1,check2
            if clusto['axis0'][a][2:4] == '[ ': 
                for b in range(1,len(clusto['axis0'][a])):
                    if clusto['axis0'][a][b] == ' ' and 'check1' not in locals():
                        check1 = b
                    elif clusto['axis0'][a][b] == ' ' and 'check2' not in locals():
                        x = float(clusto['axis0'][a][3:b])
                        check2 = b
                    elif clusto['axis0'][a][b] == ' ' and clusto['axis0'][a][b-1] != ' ' and 'check2' in locals() and 'y' not in locals():             
                        y = float(clusto['axis0'][a][check2:b])
                        z = float(clusto['axis0'][a][b:-2])
            elif clusto['axis0'][a][2] == '[' and clusto['axis0'][a][3] != ' ': 
                for b in range(1,len(clusto['axis0'][a])):
                    if clusto['axis0'][a][b] == ' ' and 'check1' not in locals():
                        check1 = b
                        x = float(clusto['axis0'][a][3:b])
                    elif clusto['axis0'][a][b] == ' ' and clusto['axis0'][a][b-1] != ' ' and 'check2' not in locals() and 'y' not in locals():# 'check2' not in locals():
                        y = float(clusto['axis0'][a][check1:b])
                        z = float(clusto['axis0'][a][b:-2])
                        check2 = b
            clust['axis0'][a] = np.array([x, y, z])
        del check1, check2, x, y, z, a, b

        # axis1
        for c in range(len(clusto["axis1"])):
            if c > 0:
                del x,y,z,check1,check2
            if clusto['axis1'][c][2:4] == '[ ': 
                for d in range(1,len(clusto['axis1'][c])):
                    if clusto['axis1'][c][d] == ' ' and 'check1' not in locals():
                        check1 = d
                    elif clusto['axis1'][c][d] == ' ' and 'check2' not in locals():
                        x = float(clusto['axis1'][c][3:d])
                        check2 = d
                    elif clusto['axis1'][c][d] == ' ' and clusto['axis1'][c][d-1] != ' ' and 'check2' in locals() and 'y' not in locals():             
                        y = float(clusto['axis1'][c][check2:d])
                        z = float(clusto['axis1'][c][d:-2])
            elif clusto['axis1'][c][2] == '[' and clusto['axis1'][c][3] != ' ': 
                for d in range(1,len(clusto['axis1'][c])):
                    if clusto['axis1'][c][d] == ' ' and 'check1' not in locals():
                        check1 = d
                        x = float(clusto['axis1'][c][3:d])
                    elif clusto['axis1'][c][d] == ' ' and clusto['axis1'][c][d-1] != ' ' and 'check2' not in locals() and 'y' not in locals():# 'check2' not in locals():
                        y = float(clusto['axis1'][c][check1:d])
                        z = float(clusto['axis1'][c][d:-2])
                        check2 = d
            clust['axis1'][c] = np.array([x, y, z])
        del check1, check2, x, y, z, c, d

        # axis2
        for e in range(len(clusto["axis2"])):
            if e > 0:
                del x,y,z,check1,check2
            if clusto['axis2'][e][2:4] == '[ ':
                for f in range(1,len(clusto['axis2'][e])):
                    if clusto['axis2'][e][f] == ' ' and 'check1' not in locals():
                        check1 = f
                    elif clusto['axis2'][e][f] == ' ' and 'check2' not in locals():
                        x = float(clusto['axis2'][e][3:f])
                        check2 = f
                    elif clusto['axis2'][e][f] == ' ' and clusto['axis2'][e][f-1] != ' ' and 'check2' in locals() and 'y' not in locals():             
                        y = float(clusto['axis2'][e][check2:f])
                        z = float(clusto['axis2'][e][f:-2])
            elif clusto['axis2'][e][2] == '[' and clusto['axis2'][e][3] != ' ':
                for f in range(1,len(clusto['axis2'][e])):
                    if clusto['axis2'][e][f] == ' ' and 'check1' not in locals():
                        check1 = f
                        x = float(clusto['axis2'][e][3:f])
                    elif clusto['axis2'][e][f] == ' ' and clusto['axis2'][e][f-1] != ' ' and 'check2' not in locals() and 'y' not in locals():# 'check2' not in locals():
                        y = float(clusto['axis2'][e][check1:f])
                        z = float(clusto['axis2'][e][f:-2])
                        check2 = f
            clust['axis2'][e] = np.array([x, y, z])

        clust_data.append(clust)
        del clust, c_data

        # Load and display points
        pymevis.OpenFile(params['c_points'][dt])

        # define mesh sdf
        from PYME.experimental.isosurface import distance_to_mesh

        mesh_sdf = lambda pts: distance_to_mesh(pts.T, mesh)

        # Load and display mesh
        from PYME.experimental._triangle_mesh import TriangleMesh
        from PYME.LMVis.layers.mesh import TriangleRenderLayer

        mesh = TriangleMesh.from_stl(params['mesh_fn'][dt])

        mesh_name = pipeline.new_ds_name('surf')
        pipeline.recipe.namespace[mesh_name] = mesh
        layer = TriangleRenderLayer(pipeline, dsname=mesh_name, method='wireframe', cmap = 'SolidCyan')
        pymevis.add_layer(layer)

        # grab the bounding box
        mesh_bbox = pymevis.glCanvas.layers[-1]._bbox

        # Load and display skeleton (this is replacing the skeleton creation within the script approach because
        # it takes so long to make the skeleton. Better to do it once and then load it in for analyses

        from PYME.experimental._triangle_mesh import TriangleMesh
        from PYME.LMVis.layers.mesh import TriangleRenderLayer

        mesh = TriangleMesh.from_stl(params['skelly'][dt])

        mesh_name = pipeline.new_ds_name('skeleton')
        pipeline.recipe.namespace[mesh_name] = mesh
        layer = TriangleRenderLayer(pipeline, dsname=mesh_name, method='wireframe', cmap = 'SolidMagenta')
        pymevis.add_layer(layer)

        # grab the bounding box
        mesh_bbox = pymevis.glCanvas.layers[-1]._bbox

        # Import ExtractTableChannel Recipe
        from PYME.recipes.localisations import ExtractTableChannel
        recipe = pipeline.recipe
        # Extract Rtn4 points as 'Rtn4' DataSource
        recipe.add_modules_and_execute([ExtractTableChannel(recipe, inputName = 'filtered_localizations', outputName = 'Rtn4',
                                                            channel = params['Rtn4_chan'])])

        # Select simulated DataSource
        pipeline.selectDataSource('Rtn4')

        # Calculate distance from simulated points to surface
        from PYME.recipes.surface_fitting import DistanceToMesh
        recipe = pipeline.recipe
        recipe.add_modules_and_execute([DistanceToMesh(recipe, input_mesh = 'surf0', input_points = 'Rtn4', output = 'Rtn4_filt')])

        # Select simulated datasource that now has the distance_to_surf0 values associated with it
        pipeline.selectDataSource('Rtn4_filt')

        # Dictionary with info about what filter to use and what values
        filt_dict_rtn = {'distance_to_surf0': [params['low_filt'],params['high_filt']]}

        # Use PYME FilterTable to filter by distance to mesh (SDF) using low_filt and high_filt variables
        from PYME.recipes.tablefilters import FilterTable
        recipe.add_modules_and_execute([FilterTable(recipe, inputName = 'Rtn4_filt', filters = filt_dict_rtn, 
                                                    outputName = 'sdf_filtered_Rtn4')])

        # Select the simulate points that passed the SDF filter above
        pipeline.selectDataSource('sdf_filtered_Rtn4')

        # Extract x,y,z coordinates for simulated points
        x_rtn = pipeline['x']
        y_rtn = pipeline['y']
        z_rtn = pipeline['z']

        # Merge x,y,z coordinates into one variable 'points_rtn'
        points_rtn = np.c_[x_rtn.ravel(),y_rtn.ravel(),z_rtn.ravel()]

        # Import cKDTree function from SciPy and use it on the skeleton
        pipeline.selectDataSource('skeleton0')
        d_skelly = pipeline.selectedDataSource
        orig_skeleton = d_skelly._vertices['position'][d_skelly._vertices['halfedge']!=-1]
        from scipy.spatial import cKDTree
        tree_orig = cKDTree(orig_skeleton)

        # Move each vertex to the average position of all of its neighbors within a 'ball_r' radius
        # Also trim out vertices that have a # of nearest neighbors > than max_nn (these are the q-tip looking spots)
        # This doesn't make the problem areas better, it just cuts them out entirely
        from collections import OrderedDict
        skeleton = orig_skeleton
        bad_inds = []
        a_ind = tree_orig.query_ball_point(skeleton, params['ball_r'])
        for trim in range(len(a_ind)):
            skeleton[trim] = np.mean(skeleton[a_ind[trim]],axis=0)
            if len(a_ind[trim]) > params['max_nn']:
                bad_inds.append(a_ind[trim])

        flat_bad_inds = [element for sublist in bad_inds for element in sublist]
        flat_unique = list(OrderedDict.fromkeys(flat_bad_inds))
        skeleton = np.delete(skeleton, flat_unique, axis=0)

        # Make KDTree based on the mean skeleton
        mean_tree_skeleton = cKDTree(skeleton)

        # Find the closest neighbor that is at least 'min_dist' away from point being queried.
        nneigh_ind = np.zeros([len(skeleton),1])
        nneigh_dist = np.zeros([len(skeleton),1])
        for aa in range(len(skeleton)):
            dist, ind = mean_tree_skeleton.query(skeleton[aa], k=len(skeleton), distance_upper_bound=params['max_dist']*2)
            for ab in range(len(dist)):
                if dist[ab] > params['min_dist']:
                    nneigh_ind[aa] = ind[ab]
                    nneigh_dist[aa] = dist[ab]
                    break

        # Calculate vectors pointing from each point to its nearest neighbor that is at least 'min_dist' away.
        skel_vecs = np.zeros([len(skeleton),3])
        bad_skel = []
        for ac in range(len(skeleton)):
            if nneigh_ind[ac] >= len(skeleton):
                bad_skel.append(ac)
            else:
                vec = skeleton[ac] - skeleton[int(nneigh_ind[ac][0])]
                skel_vecs[int(ac)] = vec
        bad_skel = np.asarray(bad_skel)
        skeleton = np.delete(skeleton, bad_skel, axis=0)
        nneigh_dist = np.delete(nneigh_dist, bad_skel, axis=0)
        nneigh_ind = np.delete(nneigh_ind, bad_skel, axis=0)
        skel_vecs = np.delete(skel_vecs, bad_skel, axis=0)

        del mean_tree_skeleton
        mean_tree_skeleton = cKDTree(skeleton)

        # Normalize
        skel_dist_mag = np.c_[nneigh_dist.ravel(),nneigh_dist.ravel(),nneigh_dist.ravel()]
        vecs_hat = skel_vecs/skel_dist_mag

        # Add those vectors to the skeleton in the GUI
        # add_ds_from_Nx3(skeleton, pipeline, pymevis, ds_name=f'nn-mean-skel-{min_dist}nm', normals=vecs_hat)

        # skel_dist = distance from each Rtn4 point to closest skeleton point; skel_ind = index of the closest point in 'skeleton'
        skel_dist_rtn, skel_ind_rtn = mean_tree_skeleton.query(points_rtn, k=1)

        ############################# Plot angles of centers of clusters to skeleton #############################

        # map centers of clusters to 'centers' as xyz coordinates
        centers = np.zeros([len(clust_data[dt]['x']),3])
        for g in range(len(clust_data[dt]['x'])):
            centers[g] = [clust_data[dt]['x'][g],clust_data[dt]['y'][g],clust_data[dt]['z'][g]]

        # determine closest skeleton point to each cluster center
        skel_dist_cent, skel_ind_cent = mean_tree_skeleton.query(centers, k=1)

        # vector from 1st nearest neighbor to Rtn cluster center
        v_cent = skeleton[skel_ind_cent]-centers

        # make skel_dist an [n, 3] array to match v_cent
        skel_dist_mag_cent = np.c_[skel_dist_cent.ravel(),skel_dist_cent.ravel(),skel_dist_cent.ravel()]

        # normalized/unit vector pointing from skeleton to Rtn cluster center point (divide each part of the vector by magnitidue of vector)
        vp_cent = v_cent/skel_dist_mag_cent

        # Calculate vector rejections to use for angle analysis
        #vecs_hat_ind = vecs_hat[skel_ind_cent] #only relevant skeleton vectors
        skel_vecs_used = vecs_hat[skel_ind_cent]
        v_cent = []
        v_cent_hat = []
        for vrej in range(len(vp_cent)):
            v_temp = vp_cent[vrej]-skel_vecs_used[vrej]*(np.dot(vp_cent[vrej],skel_vecs_used[vrej])/np.linalg.norm(skel_vecs_used[vrej]))
            if vrej == 6:
                print(v_temp)
            v_cent.append(v_temp)
            v_cent_hat_temp = v_temp/np.linalg.norm(v_temp)
            v_cent_hat.append(v_cent_hat_temp)
        v_cent = np.asarray(v_cent)
        v_cent_hat = np.asarray(v_cent_hat)

        # now let's construct the frame of reference
        # Z-direction unit vector
        zdir = np.array([0,0,1])

        # pick dir along skeleton axis as first vector in frame
        # skeleton axis vectors of skeleton points that are closest to each Rtn4 cluster center
        frame0_cent = vecs_hat[skel_ind_cent]

        # component of z not in the direction of e0 or portion of Z-dir that is orthogonal to frame0_cent
        frame1_cent = zdir[None,:] - frame0_cent*(frame0_cent*zdir[None,:]).sum(1)[:,None]

        # orthogonal to other two vectors in the frame
        frame2_cent = np.cross(frame0_cent,frame1_cent,axis=1)

        # make them unit vectors (vector divided its magnitude)
        frame1_cent_norm = frame1_cent/np.linalg.norm(frame1_cent,axis=1)[:,None]
        frame2_cent_norm = frame2_cent/np.linalg.norm(frame2_cent,axis=1)[:,None]

        # Determine angles_rtn between z-direction and 
        angles_cent = np.arccos((v_cent_hat*frame1_cent_norm).sum(1))  # this works because frame1_rtn is also normalized (unit length=1)

        # Add layer of center points with normals set as the vectors that point to closest part of the skeleton
        add_ds_from_Nx3(centers, pipeline, pymevis, 'cluster-centers', normals=clust_data[dt]['axis0'])

        # Calculate handedness of angles
        loc_vecs = v_cent_hat
        hand = np.cross(skel_vecs_used,loc_vecs)
        zhand = hand[:,2]
        hand_bool = np.ones(len(zhand))
        for neg in range(len(zhand)):
            if zhand[neg] >= 0:
                continue
            elif zhand[neg] < 0:
                hand_bool[neg] = hand_bool[neg] * -1

        angles_cent = angles_cent * hand_bool
        #del skel_vecs_used, loc_vecs, hand, zhand, hand_bool

        # Import ExtractTableChannel Recipe
        from PYME.recipes.localisations import ExtractTableChannel
        recipe = pipeline.recipe
        # Extract cluster center point

        # Select simulated DataSource
        pipeline.selectDataSource('cluster-centers')

        # Calculate distance from simulated points to surface
        from PYME.recipes.surface_fitting import DistanceToMesh
        recipe = pipeline.recipe
        recipe.add_modules_and_execute([DistanceToMesh(recipe, input_mesh = 'surf0', input_points = 'cluster-centers',
                                                       output = 'centers_sdf')])

        # Select simulated datasource that now has the distance_to_surf0 values associated with it
        pipeline.selectDataSource('centers_sdf')

        # Dictionary with info about what filter to use and what values
        filt_dict_rtn = {'distance_to_surf0': [params['low_filt'],params['high_filt']]}

        # Use PYME FilterTable to filter by distance to mesh (SDF) using low_filt and high_filt variables
        from PYME.recipes.tablefilters import FilterTable
        recipe.add_modules_and_execute([FilterTable(recipe, inputName = 'centers_sdf', filters = filt_dict_rtn, 
                                                    outputName = 'sdf_filtered_centers')])

        # Select the simulate points that passed the SDF filter above
        pipeline.selectDataSource('sdf_filtered_centers')

        # Filter data

        # Determine how far each cluster center is from the membrane surface so we can filter out those
        # that are too far away
        pipeline.selectDataSource('surf0')
        surf_d = pipeline.selectedDataSource
        surf_verts = surf_d._vertices['position'][surf_d._vertices['halfedge']!=-1]
        surf_norms = surf_d._vertices['normal']
        from scipy.spatial import cKDTree
        tree_surf = cKDTree(surf_verts)
        surf_dist, surf_ind = tree_surf.query(centers, k=1)
        skel_cent_dist, skel_cent_ind = mean_tree_skeleton.query(centers, k=1)

        # Create filters to filter out clusters that are too far from the surface and that are less than clust_min in size
        sdf_filt = surf_dist < params['high_filt']
        # min_size_filt = clust_data[dt]['count'] > params['min_size']
        # max_size_filt = clust_data[dt]['count'] < params['max_size']
        # size_filt = min_size_filt * max_size_filt
        total_filt = sdf_filt# * size_filt

        # Apply filter
        clust_data[dt]['total_filt'] = total_filt
        frame0_cent_filt = frame0_cent[total_filt]
        frame1_cent_filt = frame1_cent_norm[total_filt]
        frame2_cent_filt = frame2_cent_norm[total_filt]
        clust_filt = clust_data[dt]['axis0'][total_filt]
        centers_filt = centers[total_filt]
        angles_cent_filt = angles_cent[total_filt]
        v_cent_hat_filt = v_cent_hat[total_filt]
        skel_vecs_used_filt = skel_vecs_used[total_filt]
        ccount = clust_data[dt]['count'][total_filt]
        gr = clust_data[dt]['gyrationRadius'][total_filt]
        add_ds_from_Nx3(centers_filt, pipeline, pymevis, 'cluster-centers', normals=clust_filt)

        from numpy import pi
    # Simulated random orientations of clusters
        # 4. clusters whose axes are tangent to surface in any direction (360 degrees) and positioned as real data

        # Vectors can probably just point from each vertex to the closest vertex. That should be pretty random
        # and tangent
        # positions = cluster_center_positions
        # vectors = tangent_to_surface_and_any_direction\
        # followed advice from https://stackoverflow.com/questions/71160423/how-to-sample-points-in-3d-in-python-with-origin-and-normal-vector

        # parameters for this simulation
        r = 1 # radius of points sampled in the circle on the plane (probably leave as 1 always)
        v_num = 50 # number of vectors desired (evenly spaced around the circle)

        # equations for making the vectors
        rho = np.linspace(0, 2*np.pi, v_num)
        x = np.cos(rho) * r
        y = np.sin(rho) * r
        z = np.zeros(rho.shape)
        # positions of simulated clusters are the same as observed clusters
        arot_pos = centers_filt
        arot_tub = skel_vecs_used_filt

        # Create vectors in a circle in the plane perpendicular to the vector that points from cluster center to skeleton
        f_all = []
        for bd in range(len(arot_pos)):
            p1 = arot_pos[bd]
            n = v_cent_hat_filt[bd] # vector normal to the plane
            nabs = np.absolute(n)
            indices = (-n).argsort()[:3]
            v = np.zeros(3)
            v[indices[1]] = -n[indices[0]]
            v[indices[0]] = n[indices[1]]
            v[indices[2]] = 0
            u = np.cross(n,v)
            #normalize vectors
            v_norm = v/np.linalg.norm(v)
            u_norm = u/np.linalg.norm(u)
            f = []
            for be in range(len(rho)):
                circ_p = p1 + r * v_norm * np.cos(rho[be]) + r * u_norm * np.sin(rho[be])
                circ_vec = p1-circ_p
                f.append(circ_vec/np.linalg.norm(circ_vec))
            f = np.asarray(f)
            f_all.append(f)
        f_all = np.asarray(f_all)

        #del u
        # #Add the positions with each of the simulated cluster vectors (the number of which is equal to 'rho'-1)
        # for bf in range(len(rho)-1):
        #     name = 'all-rots-' + str(bf)
        #     add_ds_from_Nx3(arot_pos, pipeline, pymevis, name, normals=f_all[:,bf])

        # establish z-direction
        arot_z = zdir

        # Calculate angle between simulated points/vectors and tubule axis
        arot_tangs_all = []
        for bi in range(len(rho)-1):
            arot_tangs = np.zeros(len(arot_pos))
            for bg in range(len(arot_pos)):
                # Take arccos of the dot product to get the angle out
                arot_tangs[bg] = np.arccos(np.dot(arot_tub[bg],f_all[:,bi][bg]))
            # remove NaNs that occur for some reason (something about invalid value for arccos but I don't understand why)
            # it shouldn't matter though since there are so many points anyways and they are all just going to be 0 anyways
            # filter out the same ones for z-axis even though that shouldn't have issues, just to be consistent
            arot_tangs = arot_tangs[~np.isnan(arot_tangs)]
            arot_tangs_all.append(arot_tangs)
        arot_tangs_all = np.asarray(arot_tangs_all)
        flat_arot_tangs = [element for sublist in arot_tangs_all for element in sublist]
        flat_arot_tangs = np.asarray(flat_arot_tangs)
        # convert from radians to degrees
        arot_tdeg = flat_arot_tangs * (180/pi)
    # End of random orientation simulations

    ################### Calculate angle between projected Rtn4 cluster major axis and its component orthogonal to the skeleton axis ###################

    ##### New analysis using planes on the surface tangent to the skeleton and handedness
        alpha = np.zeros(len(clust_filt))
        psi = np.zeros(len(clust_filt))
        sim_alpha = np.zeros(len(clust_filt)*v_num)
        sim_psi = np.zeros(len(clust_filt)*v_num)
        for obs in range(len(clust_filt)):
            alpha[obs], psi[obs] = cluster_angles(v_cent_hat_filt[obs], clust_filt[obs], skel_vecs_used_filt[obs])
            for sims in range(v_num):
                sim_alpha[((obs*v_num)+sims)], sim_psi[((obs*v_num)+sims)] = cluster_angles(v_cent_hat_filt[obs], 
                                                                                            f_all[obs][sims],
                                                                                            skel_vecs_used_filt[obs])

        # Save a dictionary of counts, bins, and angles to cluster centers to combine experimental results later
        cluster_output = {'center_angles': angles_cent_filt, 'alpha': alpha, 'sim_alpha': sim_alpha,
                          'psi': psi, 'sim_psi': sim_psi, "count": clust_data[dt]['count'][total_filt], "x": clust_data[dt]['x'], 
                          "y": clust_data[dt]['y'][total_filt], "z": clust_data[dt]['z'][total_filt], 
                          "gyrationRadius": clust_data[dt]['gyrationRadius'][total_filt], 
                          "median_abs_deviation": clust_data[dt]['median_abs_deviation'][total_filt], 
                          "sigma0": clust_data[dt]['sigma0'][total_filt], "sigma1": clust_data[dt]['sigma1'][total_filt],
                          "sigma2": clust_data[dt]['sigma2'][total_filt], "sigma_x": clust_data[dt]['sigma_x'][total_filt], 
                          "sigma_y": clust_data[dt]['sigma_y'][total_filt], "sigma_z": clust_data[dt]['sigma_z'][total_filt], 
                          "anisotropy": clust_data[dt]['anisotropy'][total_filt], "theta": clust_data[dt]['theta'][total_filt], 
                          "phi": clust_data[dt]['phi'][total_filt], 'dbscanID': l_data[:,30]}
        cluster_output_all.append(cluster_output)
                
    ##### Simulating cluster shapes data 
        # generate positions using uniform distribution
        rng = np.random.default_rng()
        uni = rng.uniform(0,2*np.pi,10)


        cs_points_sep, cs_points_tog = tan_circ_sim(rad=gr, count=ccount,
                                                    loc_prec=np.average(clust_data[dt]['loc_prec']),
                                                    rg_factor=1, centers=centers_filt, surf_verts=surf_verts,
                                                   skeleton=skeleton,vecs_hat=vecs_hat)

        ls_points_sep, ls_points_tog = tan_line_sim(rad=gr, count=ccount,
                                                    loc_prec=np.average(clust_data[dt]['loc_prec']),
                                                    rg_factor=1, centers=centers_filt, surf_verts=surf_verts,
                                                   skeleton=skeleton,vecs_hat=vecs_hat)

        adj_points_sep, adj_points_tog = tan_line_sim(rad=gr, count=ccount,
                                                    loc_prec=np.average(clust_data[dt]['loc_prec']), 
                                                    rg_factor=rg_factor, centers=centers_filt, surf_verts=surf_verts,
                                                   skeleton=skeleton,vecs_hat=vecs_hat)

        # Measure the 3D information of each cluster and add it to a single dictionary for each simulation

        from PYME.Analysis.points import cluster_morphology

        base = np.zeros(len(ccount))
        axes = np.zeros((np.shape(clust_data[dt]['axis0'])))

        circ_clusts = {"count": base, "x": base, "y": base, "z": base, "gyrationRadius": base, 
                    "median_abs_deviation": base, "sigma0": base, "sigma1": base, "sigma2": base, 
                    "sigma_x": base, "sigma_y": base, "sigma_z": base, "anisotropy": base, 
                    "theta": base, "phi": base, "axis0": axes, "axis1": axes, "axis2": axes}

        line_clusts = {"count": base, "x": base, "y": base, "z": base, "gyrationRadius": base, 
                    "median_abs_deviation": base, "sigma0": base, "sigma1": base, "sigma2": base, 
                    "sigma_x": base, "sigma_y": base, "sigma_z": base, "anisotropy": base, 
                    "theta": base, "phi": base, "axis0": axes, "axis1": axes, "axis2": axes}
        
        adj_clusts = {"anisotropy": base}

        ### Measure simulated clusters using measure_3d

        # initialize variables
        circ_count = np.zeros(len(ccount))
        circ_x = np.zeros(len(ccount))
        circ_y = np.zeros(len(ccount))
        circ_z = np.zeros(len(ccount))
        circ_gR = np.zeros(len(ccount))
        circ_mad = np.zeros(len(ccount))
        circ_sigma0 = np.zeros(len(ccount))
        circ_sigma1 = np.zeros(len(ccount))
        circ_sigma2 = np.zeros(len(ccount))
        circ_sigmax = np.zeros(len(ccount))
        circ_sigmay = np.zeros(len(ccount))
        circ_sigmaz = np.zeros(len(ccount))
        circ_anisotropy = np.zeros(len(ccount))
        circ_theta = np.zeros(len(ccount))
        circ_phi = np.zeros(len(ccount))
        circ_axis0 = np.zeros((np.shape(clust_data[dt]['axis0'])))
        circ_axis1 = np.zeros((np.shape(clust_data[dt]['axis0'])))
        circ_axis2 = np.zeros((np.shape(clust_data[dt]['axis0'])))

        line_count = np.zeros(len(ccount))
        line_x = np.zeros(len(ccount))
        line_y = np.zeros(len(ccount))
        line_z = np.zeros(len(ccount))
        line_gR = np.zeros(len(ccount))
        line_mad = np.zeros(len(ccount))
        line_sigma0 = np.zeros(len(ccount))
        line_sigma1 = np.zeros(len(ccount))
        line_sigma2 = np.zeros(len(ccount))
        line_sigmax = np.zeros(len(ccount))
        line_sigmay = np.zeros(len(ccount))
        line_sigmaz = np.zeros(len(ccount))
        line_anisotropy = np.zeros(len(ccount))
        line_theta = np.zeros(len(ccount))
        line_phi = np.zeros(len(ccount))
        line_axis0 = np.zeros((np.shape(clust_data[dt]['axis0'])))
        line_axis1 = np.zeros((np.shape(clust_data[dt]['axis0'])))
        line_axis2 = np.zeros((np.shape(clust_data[dt]['axis0'])))
        
        adj_anisotropy = np.zeros(len(ccount))

        for id in range(len(ccount)):
        # measure circular simulated clusters and record info in circ_clusts dictionary
            circ_meas = cluster_morphology.measure_3d(x=cs_points_sep[id][:,0],y=cs_points_sep[id][:,1],
                                                      z=cs_points_sep[id][:,2])

            circ_count[id] = circ_meas['count'][0]
            circ_x[id] = circ_meas['x'][0]
            circ_y[id] = circ_meas['y'][0]
            circ_z[id] = circ_meas['z'][0]
            circ_gR[id] = circ_meas['gyrationRadius'][0]
            circ_mad[id] = circ_meas['median_abs_deviation'][0]
            circ_sigma0[id] = circ_meas['sigma0'][0]
            circ_sigma1[id] = circ_meas['sigma1'][0]
            circ_sigma2[id] = circ_meas['sigma2'][0]
            circ_sigmax[id] = circ_meas['sigma_x'][0]
            circ_sigmay[id] = circ_meas['sigma_y'][0]
            circ_sigmaz[id] = circ_meas['sigma_z'][0]
            circ_anisotropy[id] = circ_meas['anisotropy'][0]
            circ_theta[id] = circ_meas['theta'][0]
            circ_phi[id] = circ_meas['phi'][0]
            circ_axis0[id] = circ_meas['axis0'][0]
            circ_axis1[id] = circ_meas['axis1'][0]
            circ_axis2[id] = circ_meas['axis2'][0]

        # Measure linear simulated clusters and record info in line_clusts dictionary
            line_meas = cluster_morphology.measure_3d(x=ls_points_sep[id][:,0],y=ls_points_sep[id][:,1],
                                                      z=ls_points_sep[id][:,2])

            line_count[id] = line_meas['count'][0]
            line_x[id] = line_meas['x'][0]
            line_y[id] = line_meas['y'][0]
            line_z[id] = line_meas['z'][0]
            line_gR[id] = line_meas['gyrationRadius'][0]
            line_mad[id] = line_meas['median_abs_deviation'][0]
            line_sigma0[id] = line_meas['sigma0'][0]
            line_sigma1[id] = line_meas['sigma1'][0]
            line_sigma2[id] = line_meas['sigma2'][0]
            line_sigmax[id] = line_meas['sigma_x'][0]
            line_sigmay[id] = line_meas['sigma_y'][0]
            line_sigmaz[id] = line_meas['sigma_z'][0]
            line_anisotropy[id] = line_meas['anisotropy'][0]
            line_theta[id] = line_meas['theta'][0]
            line_phi[id] = line_meas['phi'][0]
            line_axis0[id] = line_meas['axis0'][0]
            line_axis1[id] = line_meas['axis1'][0]
            line_axis2[id] = line_meas['axis2'][0]
            
        # Measure adjusted linear simulated clusters and record anisotropy
            adj_meas = cluster_morphology.measure_3d(x=adj_points_sep[id][:,0],y=adj_points_sep[id][:,1],
                                                      z=adj_points_sep[id][:,2])
            
            adj_anisotropy[id] = adj_meas['anisotropy'][0]
            
        circ_clusts['count'] = np.asarray(circ_count)
        circ_clusts['x'] = np.asarray(circ_x)
        circ_clusts['y'] = np.asarray(circ_y)
        circ_clusts['z'] = np.asarray(circ_z)
        circ_clusts['gyrationRadius'] = np.asarray(circ_gR)
        circ_clusts['median_abs_deviation'] = np.asarray(circ_mad)
        circ_clusts['sigma0'] = np.asarray(circ_sigma0)
        circ_clusts['sigma1'] = np.asarray(circ_sigma1)
        circ_clusts['sigma2'] = np.asarray(circ_sigma2)
        circ_clusts['sigma_x'] = np.asarray(circ_sigmax)
        circ_clusts['sigma_y'] = np.asarray(circ_sigmay)
        circ_clusts['sigma_z'] = np.asarray(circ_sigmaz)
        circ_clusts['anisotropy'] = np.asarray(circ_anisotropy)
        circ_clusts['theta'] = np.asarray(circ_theta)
        circ_clusts['phi'] = np.asarray(circ_phi)
        circ_clusts['axis0'] = np.asarray(circ_axis0)
        circ_clusts['axis1'] = np.asarray(circ_axis1)
        circ_clusts['axis2'] = np.asarray(circ_axis2)

        line_clusts['count'] = np.asarray(line_count)
        line_clusts['x'] = np.asarray(line_x)
        line_clusts['y'] = np.asarray(line_y)
        line_clusts['z'] = np.asarray(line_z)
        line_clusts['gyrationRadius'] = np.asarray(line_gR)
        line_clusts['median_abs_deviation'] = np.asarray(line_mad)
        line_clusts['sigma0'] = np.asarray(line_sigma0)
        line_clusts['sigma1'] = np.asarray(line_sigma1)
        line_clusts['sigma2'] = np.asarray(line_sigma2)
        line_clusts['sigma_x'] = np.asarray(line_sigmax)
        line_clusts['sigma_y'] = np.asarray(line_sigmay)
        line_clusts['sigma_z'] = np.asarray(line_sigmaz)
        line_clusts['anisotropy'] = np.asarray(line_anisotropy)
        line_clusts['theta'] = np.asarray(line_theta)
        line_clusts['phi'] = np.asarray(line_phi)
        line_clusts['axis0'] = np.asarray(line_axis0)
        line_clusts['axis1'] = np.asarray(line_axis1)
        line_clusts['axis2'] = np.asarray(line_axis2)
        
        adj_clusts['anisotropy'] = np.asarray(adj_anisotropy)

        circ_sims_all.append(circ_clusts)
        line_sims_all.append(line_clusts)
        adj_sims_all.append(adj_clusts)

    save_circ_sims = params['savedir'] + '\\circular_sims_' + params['ver'] + '.npy'
    np.save(save_circ_sims, circ_sims_all, allow_pickle=True)
    
    save_line_sims = params['savedir'] + '\\linear_sims_' + params['ver'] + '.npy'
    np.save(save_line_sims, line_sims_all, allow_pickle=True)
    
    save_adj_sims = params['savedir'] + '\\adj_sims_' + params['ver'] + '.npy'
    np.save(save_adj_sims, adj_sims_all, allow_pickle=True)
    
    save_output_file = params['savedir'] + '\\cluster_output_' + params['ver'] + '.npy'
    np.save(save_output_file, cluster_output_all, allow_pickle=True)


In [ ]:
cluster_analysis_V1(params)